<a href="https://colab.research.google.com/github/thiagoigfraga/pesquisa_sisrec_interacoes/blob/main/top_k_artigos.pynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install river

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from river import forest, metrics, drift, stats
import random
import hashlib
import pickle
from datetime import datetime
import os
import traceback
from typing import Optional, Dict, Any
from collections import Counter

# Definir sementes para reprodutibilidade
random.seed(42)
np.random.seed(42)

def convert_unix_timestamp(ts):
    """Converte timestamp Unix em milissegundos para datetime"""
    return pd.to_datetime(ts, unit='ms')

def consistent_hash(value):
    """Função de hash consistente usando SHA-256"""
    return int(hashlib.sha256(str(value).encode('utf-8')).hexdigest(), 16) % 1000

class OnlineNewsRecommender:
    def __init__(
        self,
        n_models=10,
        drift_detector=drift.ADWIN(),
        top_k=10
    ):
        # Inicializa um normalizador para cada feature numérica
        self.scalers = {
            'user_id_hash': stats.Mean(),
            'article_id_hash': stats.Mean(),
            'hour_sin': stats.Mean(),
            'hour_cos': stats.Mean(),
            'day_sin': stats.Mean(),
            'day_cos': stats.Mean(),
            'month_sin': stats.Mean(),
            'month_cos': stats.Mean(),
            'hour': stats.Mean(),
            'article_popularity': stats.Mean(),
            'user_activity': stats.Mean()
        }

        self.vars = {k: stats.Var() for k in self.scalers.keys()}

        # Modelo base com detector de drift
        self.model = forest.ARFClassifier(
            n_models=n_models,
            drift_detector=drift_detector,
            grace_period=50,
            max_features='sqrt',
            seed=42,
            leaf_prediction='nb'
        )

        # Métricas online
        self.metrics = {
            'accuracy': metrics.Accuracy(),
            'f1': metrics.F1(),
            'precision': metrics.Precision(),
            'recall': metrics.Recall()
        }

        # Contadores e estado
        self.article_counter = Counter()
        self.user_counter = Counter()
        self.current_top = set()
        self.top_k = top_k

    def _normalize_feature(self, name, value):
        """Normaliza uma feature usando média e variância online"""
        # Atualiza estatísticas
        self.scalers[name].update(value)
        self.vars[name].update(value)

        # Calcula z-score
        mean = self.scalers[name].get()
        std = np.sqrt(self.vars[name].get()) if self.vars[name].get() > 0 else 1

        return (value - mean) / (std + 1e-8)

    def _extract_features(self, x):
        """
        Extrai as features do exemplo de entrada.

        Parameters:
        -----------
        x : dict
            Dicionário contendo os dados do exemplo atual.

        Returns:
        --------
        dict
            Dicionário com as features normalizadas.
        """
        timestamp = convert_unix_timestamp(x['click_timestamp'])

        # Features básicas
        features = {
            'user_id_hash': consistent_hash(x['user_id']),
            'article_id_hash': consistent_hash(x['click_article_id']),
            'hour_sin': np.sin(2 * np.pi * timestamp.hour / 24),
            'hour_cos': np.cos(2 * np.pi * timestamp.hour / 24),
            'day_sin': np.sin(2 * np.pi * timestamp.dayofweek / 7),
            'day_cos': np.cos(2 * np.pi * timestamp.dayofweek / 7),
            'month_sin': np.sin(2 * np.pi * timestamp.month / 12),
            'month_cos': np.cos(2 * np.pi * timestamp.month / 12),
            'hour': timestamp.hour,
        }

        # Calcula popularidade excluindo a interação atual
        article_count = self.article_counter.get(x['click_article_id'], 0)
        user_count = self.user_counter.get(x['user_id'], 0)

        total_interactions = sum(self.article_counter.values()) or 1
        total_users = sum(self.user_counter.values()) or 1

        features.update({
            'article_popularity': article_count / total_interactions,
            'user_activity': user_count / total_users,
        })

        # Normaliza features
        normalized_features = {
            name: self._normalize_feature(name, value)
            for name, value in features.items()
            if name in self.scalers
        }

        # Adiciona features categóricas sem normalização
        normalized_features['weekend'] = int(timestamp.dayofweek >= 5)

        return normalized_features

    def learn_one(self, x):
        """Aprende com um exemplo"""
        # Atualiza top articles antes de processar a interação atual para evitar vazamento
        if sum(self.article_counter.values()) % 100 == 0:
            self.current_top = set(
                article for article, _ in
                self.article_counter.most_common(self.top_k)
            )

        # Define target (1 se artigo está no top-k)
        target = int(x['click_article_id'] in self.current_top)

        # Atualiza contadores após definir o target
        self.article_counter[x['click_article_id']] += 1
        self.user_counter[x['user_id']] += 1

        # Extrai features
        features = self._extract_features(x)

        # Treina o modelo
        self.model.learn_one(features, target)

        # Atualiza métricas
        pred = self.model.predict_one(features)
        if pred is not None:
            for metric in self.metrics.values():
                metric.update(target, pred)

        return self

    def predict_one(self, x):
        """Faz predição para um exemplo"""
        features = self._extract_features(x)
        return self.model.predict_one(features)

    def predict_proba_one(self, x):
        """Retorna probabilidades para um exemplo"""
        features = self._extract_features(x)
        return self.model.predict_proba_one(features)

    def save(self, path):
        """Salva o modelo completo"""
        with open(path, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, path):
        """Carrega o modelo completo"""
        with open(path, 'rb') as f:
            return pickle.load(f)


class NewsRecommenderSystem:
    """Sistema para gerenciar treinamento e predição do recomendador de notícias."""

    def __init__(self, mode='train', model_path=None):
        """
        Inicializa o sistema de recomendação.

        Parameters:
        -----------
        mode : str
            'train' para treinar novo modelo, 'predict' para usar modelo existente.
        model_path : str, optional
            Caminho para o modelo existente quando mode='predict'.
        """
        self.mode = mode
        self.model = None
        self.history = {
            'processed_records': 0,
            'start_time': datetime.now(),
            'metrics': {},
            'dataset_info': {},
            'model_info': {}
        }

        if mode == 'predict' and model_path:
            self._load_existing_model(model_path)
        else:
            self._initialize_new_model()

    def _load_existing_model(self, model_path):
        """Carrega modelo existente para predição."""
        try:
            self.model = OnlineNewsRecommender.load(model_path)
            self.history['model_info'] = {
                'original_model_path': model_path,
                'load_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        except Exception as e:
            raise Exception(f"Erro ao carregar modelo: {str(e)}")

    def _initialize_new_model(self):
        """Inicializa novo modelo para treinamento."""
        self.model = OnlineNewsRecommender(
            n_models=15,
            drift_detector=drift.ADWIN(delta=0.001),
            top_k=10
        )
        self.history['model_info'] = {
            'initialization_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'model_type': 'OnlineNewsRecommender',
            'n_models': 15,
            'drift_detector': 'ADWIN',
            'top_k': 10
        }

    def validate_dataset(self, dataset: pd.DataFrame) -> bool:
        """
        Valida se o dataset tem as colunas necessárias.

        Parameters:
        -----------
        dataset : pd.DataFrame
            O dataset a ser validado.

        Returns:
        --------
        bool
            True se o dataset é válido, levanta ValueError caso contrário.
        """
        required_columns = ['user_id', 'click_article_id', 'click_timestamp']
        missing_columns = [col for col in required_columns if col not in dataset.columns]

        if missing_columns:
            raise ValueError(f"Colunas faltando no dataset: {missing_columns}")

        return True

    def _update_final_metrics(self):
        """
        Atualiza métricas finais no histórico com informações adicionais.
        """
        if self.mode == 'train':
            n_drifts = sum(self.model.model[n].drift_detector.n_detections
                           for n in range(self.model.model.n_models))
            n_warnings = sum(self.model.model[n].drift_detector.n_warnings
                             for n in range(self.model.model.n_models))
            top_articles = dict(self.model.article_counter.most_common(10))
        else:
            n_drifts = n_warnings = None
            top_articles = None

        self.history['metrics'] = {
            'final_accuracy': self.model.metrics['accuracy'].get() if self.mode == 'train' else None,
            'final_f1': self.model.metrics['f1'].get() if self.mode == 'train' else None,
            'final_precision': self.model.metrics['precision'].get() if self.mode == 'train' else None,
            'final_recall': self.model.metrics['recall'].get() if self.mode == 'train' else None,
            'processed_records': self.history['processed_records'],
            'processing_time': str(datetime.now() - self.history['start_time']),
            'n_drifts': n_drifts,
            'n_warnings': n_warnings,
            'top_articles': str(top_articles) if top_articles else None
        }

    def analyze_samples(self, dataset):
        """
        Analisa e mostra informações sobre as amostras do dataset com validações adicionais.

        Parameters:
        -----------
        dataset : pd.DataFrame
            O dataset a ser analisado.

        Returns:
        --------
        pd.DataFrame
            O dataset possivelmente limpo.
        """
        try:
            print("\n=== Análise das Amostras ===")

            # Verifica duplicações
            duplicates = dataset.duplicated().sum()
            if duplicates > 0:
                print(f"Aviso: Encontradas {duplicates:,} linhas duplicadas")
                print("Removendo duplicatas...")
                dataset = dataset.drop_duplicates()

            # Contagem de registros
            total_samples = len(dataset)
            total_interactions = dataset.groupby(['user_id', 'click_article_id', 'click_timestamp']).size().sum()

            if total_samples != total_interactions:
                print(f"Aviso: Possível inconsistência na contagem de amostras")
                print(f"Total de linhas: {total_samples:,}")
                print(f"Total de interações únicas: {total_interactions:,}")

            print(f"Total de amostras: {total_interactions:,}")
            print(f"Período: {dataset['timestamp'].min()} até {dataset['timestamp'].max()}")
            print(f"Usuários únicos: {dataset['user_id'].nunique():,}")
            print(f"Artigos únicos: {dataset['click_article_id'].nunique():,}")

            # Verificar integridade dos dados
            print("\nVerificação de integridade:")
            null_counts = dataset.isnull().sum()
            if null_counts.any():
                print("Valores nulos encontrados:")
                print(null_counts[null_counts > 0])
                # Tratar ou remover valores nulos conforme apropriado
                dataset = dataset.dropna()

            # Análise de timestamps
            invalid_timestamps = pd.to_datetime(dataset['click_timestamp'], unit='ms', errors='coerce').isnull().sum()
            if invalid_timestamps > 0:
                print(f"\nAviso: {invalid_timestamps} timestamps inválidos encontrados")
                # Tratar timestamps inválidos
                dataset = dataset.dropna(subset=['click_timestamp'])

            # Análises adicionais podem ser adicionadas aqui...

            return dataset  # Retorna dataset possivelmente limpo

        except Exception as e:
            print(f"Erro analisando amostras: {str(e)}")
            traceback.print_exc()
            return dataset

    def process_dataset(self, dataset, batch_size=1000, report_interval=10000, output_model_path=None):
        """
        Processa o dataset, treinando ou prevendo com base no modo.

        Parameters:
        -----------
        dataset : pd.DataFrame
            O dataset a ser processado.
        batch_size : int
            Número de registros a processar por lote.
        report_interval : int
            Intervalo para reportar progresso.
        output_model_path : str, optional
            Caminho para salvar o modelo atualizado.

        Returns:
        --------
        dict
            O histórico do processamento.
        """
        try:
            self.validate_dataset(dataset)

            # Análise das amostras
            print("\nAnalisando amostras...")
            dataset['timestamp'] = pd.to_datetime(
                dataset['click_timestamp'],
                unit='ms',
                errors='coerce'
            )
            dataset = self.analyze_samples(dataset)

            self.history['dataset_info'] = {
                'total_records': len(dataset),
                'unique_users': dataset['user_id'].nunique(),
                'unique_articles': dataset['click_article_id'].nunique(),
                'start_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'batch_size': batch_size,
                'report_interval': report_interval
            }

            # Ordena o dataset pelo timestamp
            dataset = dataset.sort_values('timestamp').reset_index(drop=True)

            # Inicializa lista de predições se em modo de predição
            predictions = []

            for start_idx in range(0, len(dataset), batch_size):
                try:
                    batch = dataset.iloc[start_idx:start_idx + batch_size]

                    for _, row in batch.iterrows():
                        if self.mode == 'train':
                            # Aprende com o exemplo
                            self.model.learn_one(row.to_dict())
                        else:
                            # Faz predição e armazena
                            prediction = self.model.predict_one(row.to_dict())
                            predictions.append({
                                'user_id': row['user_id'],
                                'click_article_id': row['click_article_id'],
                                'prediction': prediction
                            })
                        self.history['processed_records'] += 1

                    if (start_idx + batch_size) % report_interval == 0:
                        self._report_progress()

                except Exception as e:
                    print(f"Erro processando batch {start_idx}-{start_idx+batch_size}: {str(e)}")
                    traceback.print_exc()
                    continue

            if self.mode == 'predict' and predictions:
                predictions_df = pd.DataFrame(predictions)
                predictions_df.to_csv('predictions.csv', index=False)
                print("Predições salvas em 'predictions.csv'")

            if output_model_path:
                self.save_model(output_model_path)

            self._update_final_metrics()
            return self.history

        except Exception as e:
            print(f"Erro fatal processando dataset: {str(e)}")
            traceback.print_exc()
            return None

    def _report_progress(self):
        """Reporta progresso do processamento."""
        print(f"\nProcessados {self.history['processed_records']} exemplos")

        if self.mode == 'train':
            print("\nMétricas atuais:")
            for name, metric in self.model.metrics.items():
                print(f"{name}: {metric.get():.4f}")

            n_drifts = sum(self.model.model[n].drift_detector.n_detections
                           for n in range(self.model.model.n_models))
            print(f"\nDrifts detectados: {n_drifts}")

    def save_model(self, path):
        """Salva o modelo."""
        self.model.save(path)
        self.history['model_info']['final_model_path'] = path
        self.history['model_info']['save_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def save_history(self):
        """
        Versão melhorada do salvamento de histórico.
        """
        try:
            # Prepara dados para o CSV
            history_df = pd.DataFrame([{
                'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'mode': self.mode,
                'dataset_size': self.history['dataset_info'].get('total_records'),
                'unique_users': self.history['dataset_info'].get('unique_users'),
                'unique_articles': self.history['dataset_info'].get('unique_articles'),
                'batch_size': self.history['dataset_info'].get('batch_size'),
                'processed_records': self.history['metrics'].get('processed_records'),
                'accuracy': self.history['metrics'].get('final_accuracy'),
                'f1_score': self.history['metrics'].get('final_f1'),
                'precision': self.history['metrics'].get('final_precision'),
                'recall': self.history['metrics'].get('final_recall'),
                'processing_time': self.history['metrics'].get('processing_time'),
                'n_drifts': self.history['metrics'].get('n_drifts'),
                'n_warnings': self.history['metrics'].get('n_warnings'),
                'top_articles': self.history['metrics'].get('top_articles'),
                'model_path': self.history['model_info'].get('final_model_path'),
                'original_model': self.history['model_info'].get('original_model_path')
            }])

            # Define arquivo apropriado
            filename = 'training_history.csv' if self.mode == 'train' else 'prediction_history.csv'

            # Garante que o diretório existe
            os.makedirs(os.path.dirname(filename) if os.path.dirname(filename) else '.', exist_ok=True)

            # Verifica se arquivo existe
            if os.path.exists(filename):
                existing_df = pd.read_csv(filename)
                history_df = pd.concat([existing_df, history_df], ignore_index=True)

            history_df.to_csv(filename, index=False)
            print(f"Histórico salvo em {filename}")

        except Exception as e:
            print(f"Erro salvando histórico: {str(e)}")
            traceback.print_exc()

def main(mode: str = 'train',
         input_model: Optional[str] = None,
         dataset_path: Optional[str] = None,
         output_model: Optional[str] = None) -> Optional[Dict[str, Any]]:
    """
    Função principal para executar o sistema de recomendação.

    Parameters:
    -----------
    mode : str
        Modo de operação: 'train' ou 'predict'.
    input_model : str, optional
        Caminho do modelo para modo 'predict'.
    dataset_path : str
        Caminho do arquivo CSV com os dados.
    output_model : str, optional
        Caminho onde salvar o modelo atualizado.

    Returns:
    --------
    dict
        Histórico do processamento ou None em caso de erro.
    """
    try:
        # Validações
        if mode not in ['train', 'predict']:
            raise ValueError("Mode deve ser 'train' ou 'predict'")

        if mode == 'predict' and not input_model:
            raise ValueError("input_model é obrigatório no modo 'predict'")

        if not dataset_path:
            raise ValueError("dataset_path é obrigatório")

        # Inicializa sistema
        system = NewsRecommenderSystem(mode=mode, model_path=input_model)

        # Carrega dataset
        print(f"Carregando dataset de {dataset_path}...")
        dataset = pd.read_csv(dataset_path)

        # Processa dataset
        history = system.process_dataset(
            dataset=dataset,
            batch_size=1000,
            report_interval=50000,
            output_model_path=output_model
        )

        # Salva histórico
        system.save_history()

        return history

    except Exception as e:
        print(f"Erro na execução principal: {str(e)}")
        traceback.print_exc()
        return None

if __name__ == "__main__":
    # Configuração exemplo
    history = main(
        mode='predict',
        input_model='/content/drive/MyDrive/Pesquisa2024/models/news_recommender_new2.pkl',
        dataset_path='/content/drive/MyDrive/Pesquisa2024/dataset_interacoes.csv',
        output_model='news_recommender_new3.pkl'
    )

    # Uso para treinamento
    '''
    main(
        mode='train',
        dataset_path='dataset_treino.csv',
        output_model='modelo_treinado.pkl'
    )
    '''

    if history:
        print("\nProcessamento concluído com sucesso!")

        # Mostra estatísticas finais
        print("\n=== Estatísticas Finais ===")
        print(f"Total de registros processados: {history['processed_records']:,}")
        print(f"Tempo total de processamento: {history['metrics']['processing_time']}")

        if history['metrics'].get('n_drifts'):
            print(f"Drifts detectados: {history['metrics']['n_drifts']}")

        # Mostra top artigos se disponível
        if history['metrics'].get('top_articles'):
            print("\nTop Artigos:")
            top_articles = eval(history['metrics']['top_articles'])
            for article, count in top_articles.items():
                print(f"Artigo {article}: {count:,} interações")

Carregando dataset de /content/drive/MyDrive/Pesquisa2024/dataset_interacoes.csv...

Analisando amostras...

=== Análise das Amostras ===
Aviso: Encontradas 153,943 linhas duplicadas
Removendo duplicatas...
Aviso: Possível inconsistência na contagem de amostras
Total de linhas: 2,988,182
Total de interações únicas: 2,988,181
Total de amostras: 2,988,181
Período: 2017-10-01 03:00:00.026000 até 2017-11-13 20:04:14.886000
Usuários únicos: 322,897
Artigos únicos: 46,034

Verificação de integridade:
Valores nulos encontrados:
click_timestamp    1
timestamp          1
dtype: int64

Processados 50000 exemplos

Processados 100000 exemplos

Processados 150000 exemplos

Processados 200000 exemplos

Processados 250000 exemplos

Processados 300000 exemplos

Processados 350000 exemplos
